In [16]:
import os
import glob
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

#불러오기,결측값제거
df = pd.read_csv('./contest.csv',sep=',', error_bad_lines=False)
df = df.dropna()

#시상규모 범주분포화
df['시상규모(만원)'] = df['rewards'].apply(lambda x: (x.replace('억', '0000')))          #1억 = 10000만원으로 변경
df["시상규모(만원)"]=df["시상규모(만원)"].str.extract(r'(\d+)')                           #숫자만남기기
df["시상규모(만원)"] = df["시상규모(만원)"].fillna(0).astype(int)                  #NaN(미정)값 0으로 변경                   
df=df.astype({"시상규모(만원)":'int'})
prize_bins=[0,1,251,551,280000000]                        #범주화  0:인원미정/1-560(만원):소/561-1075(만원):중/1076-280000000(만원):대
prize_scale=["시상규모미정","소","중","대"]               
df["시상 규모"]=pd.cut(df["시상규모(만원)"],bins=prize_bins,right=False,labels=prize_scale) #분포범주화



#공모분야 더미변수화
df['fields'] = df['fields'].replace('기타', '공모분야_기타')                                                 #'기타'열이 중복됨으로 교체 
df = df.join(df["fields"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#기업형태 더미변수화                                         
df['company_type'] = df['company_type'].replace('기타', '기업형태_기타')                                                 #'기타'열이 중복됨으로 교체 
df = df.join(df["company_type"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#참여대상 더미변수화
df = df.join(df["target"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#활동혜택 더미변수화
df["contest_benefit"] = df["contest_benefit"].replace('기타', 'benefit_기타',regex=True) 
df = df.join(df["contest_benefit"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())



#csv파일저장
df=df.to_csv('contest_process.csv')

C:\Users\365me\AppData\Local\Temp\ipykernel_281396\9527536.py:10: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('./contest.csv',sep=',', error_bad_lines=False)
b'Skipping line 3036: expected 9 fields, saw 17\n'
